In [19]:
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout, Flatten, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import pandas as pd
import numpy as np
import tensorflow 
from sklearn import metrics
from tensorflow.keras import Model

In [20]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [21]:
#preprocessing 
X_train = X_train.reshape((X_train.shape[0],784)).astype('float32')
X_test = X_test.reshape((X_test.shape[0],784)).astype('float32')
print(X_train.shape, X_test.shape)


y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

#normalization
X_train = X_train/255
X_test = X_test/255

(60000, 784) (10000, 784)


In [22]:
# Vanilla Autoencoder 
input_size = 784
hidden_size = 150
output_size = 784

in1 = Input(shape=(input_size,))
hidd1 = Dense(hidden_size, activation='sigmoid')(in1)
res1 = Dense(output_size, activation='sigmoid')(hidd1)

auto1 = Model(inputs = in1, outputs = res1)
auto1.summary()
auto1.compile(loss = 'mse', optimizer='adam', metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_8 (Dense)              (None, 150)               117750    
_________________________________________________________________
dense_9 (Dense)              (None, 784)               118384    
Total params: 236,134
Trainable params: 236,134
Non-trainable params: 0
_________________________________________________________________


In [23]:
auto1.compile(loss="mean_squared_error",optimizer='adam',metrics=['accuracy'])
auto1.fit(X_train,X_train,batch_size = 128,  epochs=10)

Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0751 - accuracy: 0.0149
Epoch 2/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0313 - accuracy: 0.0109
Epoch 3/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0203 - accuracy: 0.0109
Epoch 4/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0146 - accuracy: 0.0118
Epoch 5/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0113 - accuracy: 0.0133
Epoch 6/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0092 - accuracy: 0.0137
Epoch 7/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0078 - accuracy: 0.0133
Epoch 8/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0068 - accuracy: 0.0132
Epoch 9/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0060 - accuracy: 0.0135
Epoch 10/10
469/469 [==============================] - 1s 3ms/step - loss: 0.0053 - accuracy: 0.0142

In [24]:
#info about model parameters
print(type(auto1.layers[1].get_weights()[0]))
print(type(auto1.layers[1].get_weights()[1]))
print(auto1.layers[1].get_weights()[0].shape)
print(auto1.layers[1].get_weights()[1].shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(784, 150)
(150,)


# Making Stacked Auto Encoder

In [25]:

auto1_hidden = auto1.layers[1].output
auto1_trimmed = Model(inputs = in1, outputs = auto1_hidden)
X_train_2 = auto1_trimmed.predict(X_train)
X_test_2 = auto1_trimmed.predict(X_test) 

In [26]:
x_2 = Input(shape=(150,))
h_2 = Dense(units=75, activation='sigmoid')(x_2)
r_2 = Dense(units=150, activation='sigmoid')(h_2)

auto_2 = Model(inputs= x_2, outputs= r_2)
auto_2.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
auto_2.fit(X_train_2,X_train_2,epochs=5,verbose=1)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0729 - accuracy: 0.0598
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0215 - accuracy: 0.1328
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0116 - accuracy: 0.1696
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0084 - accuracy: 0.2079
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0068 - accuracy: 0.2284


In [27]:
auto_2_hidden = auto_2.layers[1].output
auto_2_trimmed = Model(inputs = x_2, outputs = auto_2_hidden)
X_train_3 = auto_2_trimmed.predict(X_train_2)
X_test_3 = auto_2_trimmed.predict(X_test_2) 
print(X_train_3.shape)

(60000, 75)


In [28]:
x_3 = Input(shape=(75,))
r_3 = Dense(units=10, activation='softmax')(x_3)

classifier = Model(inputs = x_3, outputs = r_3)
classifier.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(classifier.layers[1].get_weights()[0].shape)
classifier.fit(X_train_3, y_train, epochs=5)



(75, 10)
Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 1.7886 - accuracy: 0.4955
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.7610 - accuracy: 0.8500
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5456 - accuracy: 0.8733
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4617 - accuracy: 0.8819
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4191 - accuracy: 0.8883


In [29]:
print(classifier.evaluate(X_test_3, y_test))

313/313 [==============================] - 1s 2ms/step - loss: 0.3766 - accuracy: 0.9000
[0.37662988901138306, 0.8999999761581421]


In [30]:
stack_auto = Sequential()
stack_auto.add(auto1.layers[0])
stack_auto.add(auto1.layers[1])
stack_auto.add(auto_2.layers[1])
stack_auto.add(classifier.layers[-1])

In [31]:
stack_auto.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(stack_auto.layers[1].get_weights()[0].shape)
print(auto_2.layers[1].get_weights()[0].shape)
stack_auto.evaluate(X_test, y_test)

(150, 75)
(150, 75)
313/313 [==============================] - 1s 2ms/step - loss: 0.4257 - accuracy: 0.8845


[0.37662988901138306, 0.8999999761581421]